### Build Calgary Road Graph and save

In [1]:
import osmnx as ox
import networkx as nx

# Define the location as Calgary, Canada
location = "Calgary, Canada"

# Download the street network for the location
graph = ox.graph_from_place(location, network_type="drive")


### Build Calgary Extreme Weather Graph and save

In [2]:
calgary_slope = ox.io.load_graphml('network.graphml')

In [3]:
# Iterate over the edges in the graph and multiply their length with grade_abs
lenght_dict = nx.get_edge_attributes(calgary_slope, 'length')
grade_abs_dict = nx.get_edge_attributes(calgary_slope, 'grade_abs')

for u, v, key, data in calgary_slope.edges(keys=True, data=True):
    if (u, v) in calgary_slope.edges():
        length = float( lenght_dict[(u, v, key)])
        grade_abs = float(grade_abs_dict[( u, v, key)])
        if grade_abs == 0.000:
            grade_abs = 0.0001
        data['slopized_length'] = length * grade_abs
        calgary_slope.add_edge(u, v, key, **data)

### Build Calgary Gas Emmision Graph and save

In [5]:
import pandas as pd
calgary_trees = pd.read_csv('Public_Trees.csv')

C:\Users\Hadia\AppData\Local\Temp\ipykernel_15800\2841489650.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  calgary_trees = pd.read_csv('Public_Trees.csv')


In [6]:
tree_lats = []
tree_longs = []
for ro in range(len(calgary_trees)):
    coordinates = calgary_trees.loc[ro,'POINT'][6:]
    # extract the latitude and longitude values as strings
    long_str, lat_str = coordinates.strip('()').split()
    tree_lats.append(float(lat_str))
    tree_longs.append(float(long_str))

In [21]:
import osmnx as ox
import geopandas as gpd
import numpy as np

# Create a GeoDataFrame of the trees with their latitudes and longitudes
trees = gpd.GeoDataFrame(geometry=gpd.points_from_xy(tree_lats,tree_longs))

# Find the nearest nodes in the graph for each tree location
tree_nodes = []
tree_edges = []
for ro in range(len(tree_lats)):
    tree_nodes.append(ox.distance.nearest_nodes(calgary_slope, tree_longs[ro], tree_lats[ro], return_dist=False))
    #tree_edges.append(ox.distance.nearest_edges(calgary_slope, tree_longs[ro], tree_lats[ro], return_dist=False))
    if ro %10000 == 0:
        print(ro)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000


In [22]:
import pickle
# save the list to a file
with open('./mapGraphs/tree_nodes.pkl', 'wb') as f:
    pickle.dump(tree_nodes, f)

In [23]:
# Create a dictionary that maps each node to the number of trees near it
tree_counts = dict(zip(*np.unique(tree_nodes, return_counts=True)))


In [25]:
# Add the tree count as an attribute to each node in the graph
for node, count in tree_counts.items():
    calgary_slope.nodes[node]['tree_count'] = count

In [26]:
# Iterate through the edges of the graph
for u, v, key, data in calgary_slope.edges(keys=True, data=True):
    
    # Count the total number of trees near the two connected nodes of the edge
    tree_count_u = calgary_slope.nodes[u].get('tree_count', 0)
    tree_count_v = calgary_slope.nodes[v].get('tree_count', 0)
    edge_tree_count = tree_count_u + tree_count_v
    
    # Add the 'edge_tree_count' attribute to the edge
    data['edge_tree_count'] = edge_tree_count
    calgary_slope.add_edge(u, v, key, **data)

In [27]:
nodes_gdf, edges_gdf = ox.graph_to_gdfs(calgary_slope)

In [ ]:
# Convert edge data to a pandas dataframe
edge_data = ox.graph_to_gdfs(calgary_slope, nodes=False, edges=True)

# Convert slopized_length to float and edge_tree_count to int
edge_data['slopized_length'] = edge_data['slopized_length'].astype(float)
edge_data['edge_tree_count'] = edge_data['edge_tree_count'].astype(int)

# Replace any zeros with the desired values
edge_data['slopized_length'] = edge_data['slopized_length'].replace(0, 0.005)
edge_data['edge_tree_count'] = edge_data['edge_tree_count'].replace(0, 1)

# Update the edge data in the graph
for u, v, key in calgary_slope.edges(keys=True):
    if key in edge_data.index and u == edge_data.at[key, 'u'] and v == edge_data.at[key, 'v']:
        calgary_slope[u][v][key]['slopized_length'] = edge_data.at[key, 'slopized_length']
        calgary_slope[u][v][key]['edge_tree_count'] = edge_data.at[key, 'edge_tree_count']




In [ ]:
# Convert edge data to a pandas dataframe
edge_data = ox.graph_to_gdfs(calgary_slope, nodes=False, edges=True)

edge_data = edge_data.reset_index()
for i in range(len(edge_data)):
    u = edge_data.iloc[i]['u']
    v = edge_data.iloc[i]['v']
    key = edge_data.iloc[i]['key']
    sl_l = float(edge_data.iloc[i]['slopized_length']) if float(edge_data.iloc[i]['slopized_length']) > 0.0 else 0.005
    ed_tr = int(edge_data.iloc[i]['edge_tree_count']) if int(edge_data.iloc[i]['edge_tree_count']) > 0 else 1
    calgary_slope[u][v][key]['slopized_length'] = sl_l
    calgary_slope[u][v][key]['edge_tree_count'] = 1000 - ed_tr

### Works:

In [ ]:
nodes_gdf, edges_gdf = ox.graph_to_gdfs(calgary_slope)
edges_gdf['edge_tree_count'] = edges_gdf['edge_tree_count'].astype(int)
edges_gdf['slopized_length'] = edges_gdf['slopized_length'].astype(float)

In [ ]:
edges_gdf.dtypes

In [ ]:
# convert the pair of GeoDataFrames to a networkx graph object
calgary_slope = ox.graph_from_gdfs(nodes_gdf, edges_gdf)

In [ ]:
# Update the edge data in the graph
ox.io.save_graphml(calgary_slope, './mapGraphs/calgary_drive_network_updated.graphml')

In [29]:
# Save the graph as a GraphML file
#ox.save_graphml(calgary_slope, filepath='./mapGraphs/calgary_drive_network.graphml')